In [15]:
from utils import read_lines
import re

from functools import cache

    
class BluePrint:
    def __init__(self, id, ore_bot, clay_bot, obsidian_bot, geode_bot):
        self.id = id
        self.ore_bot = ore_bot
        self.clay_bot = clay_bot
        self.obsidian_bot = obsidian_bot
        self.geode_bot = geode_bot
        self.max_ore_bot = max(self.clay_bot, self.obsidian_bot[0], self.geode_bot[0])
        self.max_clay_bot = self.obsidian_bot[1]
        self.max_obsidian_bot = self.geode_bot[1]
        self.limit = 24
        self.cur_max = 0

    def __repr__(self):
        return f'BluePrint({self.id=}, {self.ore_bot=}, {self.clay_bot=}, {self.obsidian_bot=}, {self.geode_bot=})'
    
    def theoritical_max(self, resources, bots, day):
        return resources[3] + bots[3] * (self.limit - day) + sum(range(1, self.limit - day))
    
    @cache
    def recur(self, resources, bots, day, ignored=()):
        if day == self.limit:
            self.cur_max = max(self.cur_max, resources[3])
            return
        
        if self.theoritical_max(resources, bots, day) < self.cur_max:
            return
        if resources[0] >= self.geode_bot[0] and resources[2] >= self.geode_bot[1]:
            resources1 = [resources[0] - self.geode_bot[0], resources[1], resources[2] - self.geode_bot[1], resources[3]]
            resources1 = tuple([resources1[i] + bots[i] for i in range(4)])
            bots1 = (bots[0], bots[1], bots[2], bots[3] + 1)
            return self.recur(resources1, bots1, day + 1)
        
        
        options = []
        if day >= self.limit - 1:
            options = []
        else:
            if bots[2] < self.max_obsidian_bot and resources[0] >= self.obsidian_bot[0] and resources[1] >= self.obsidian_bot[1] and 'obsidian_bot' not in ignored:
                options.append('obsidian_bot')
            elif not bots[2] and bots[0] < self.max_ore_bot and bots[1] < 4 and resources[0] >= self.ore_bot and 'ore_bot' not in ignored:
                options.append('ore_bot')
            if not bots[3] and bots[1] < self.max_clay_bot and bots[2] < 4 and resources[0] >= self.clay_bot and 'clay_bot' not in ignored:
                options.append('clay_bot')
            

        resources1 = [resources[i] + bots[i] for i in range(4)]
    
        self.recur(tuple(resources1), bots, day + 1, tuple(options))  # produce no bot

        for opt in options:
            if opt == 'obsidian_bot':
                resources2 = (resources1[0] - self.obsidian_bot[0], resources1[1] - self.obsidian_bot[1], resources1[2], resources1[3])
                bots1 = (bots[0], bots[1], bots[2] + 1, bots[3])
                self.recur(resources2, bots1, day + 1)
            elif opt == 'clay_bot':
                resources2 = (resources1[0] - self.clay_bot, resources1[1], resources1[2], resources1[3])
                bots1 = (bots[0], bots[1] + 1, bots[2], bots[3])
                self.recur(resources2, bots1, day + 1)
            elif opt == 'ore_bot':
                resources2 = (resources1[0] - self.ore_bot, resources1[1], resources1[2], resources1[3])
                bots1 = (bots[0] + 1, bots[1], bots[2], bots[3])
                self.recur(resources2, bots1, day + 1)


re_line = re.compile(r'.* (\d+):.* (\d+) .* (\d+) .* (\d+) .* (\d+) .* (\d+) .* (\d+) .*')
def parse_line(line):
    m = re_line.match(line)
    gs = [int(x) for x in m.groups()]
    return BluePrint(gs[0], gs[1], gs[2], (gs[3], gs[4]), (gs[5], gs[6]))

class Game:
    def __init__(self, blue_print):
        self.blue_print = blue_print
        self.resources = [0] * 4
        self.bots = [0] * 4

def part1(input_file):
    lines = read_lines(input_file)
    ans = 0
    for line in lines:
        bp = parse_line(line)
        bp.limit = 24
        bp.recur((0, 0, 0, 0), (1, 0, 0, 0), 0)
        ans += bp.id * bp.cur_max
    return ans

def part2(input_file):
    lines = read_lines(input_file)[:3]
    ans = 1
    for line in lines:
        bp = parse_line(line)
        bp.limit = 32
        bp.recur((0, 0, 0, 0), (1, 0, 0, 0), 0)
        ans *= bp.cur_max
    return ans

In [12]:
lines = read_lines('inputs/day19_test.txt')
bp = parse_line(lines[0])
bp.recur((0, 0, 0, 0), (1, 0, 0,0), 0)
bp.cur_max

9

In [13]:
part1("inputs/day19_test.txt")

33

In [10]:
part1("inputs/day19.txt")

1659

In [17]:
part2("inputs/day19_test.txt")

3472

In [16]:
part2("inputs/day19.txt")

6804